In [3]:
import json
from dataclasses import dataclass
from typing import Optional, List

@dataclass
class PatientData:
    question1: Optional[List[str]] = None  # Screening selections
    question2: Optional[int] = None  # Age
    question15: Optional[str] = None  # Family history of glaucoma
    question5: Optional[str] = None   # Diabetes status
    question24: Optional[str] = None  # Taking blood pressure medication (proxy for hypertension)

def analyze_glaucoma_screening_indication(patient_data: PatientData) -> str:
    """
    Analyzes patient data to determine glaucoma screening indication and strategy
    based on provided guidelines for average-risk patients, incorporating hypertension and diabetes.

    Args:
        patient_data: PatientData object containing relevant questionnaire responses.

    Returns:
        A string indicating if glaucoma screening is recommended, and if so,
        the modality and frequency. Returns a message about missing data if necessary.
    """

    if patient_data.question1 is None or "Item 12" not in patient_data.question1:
        return "Patient is not indicated for glaucoma screening as it was not selected in requested screening."

    if patient_data.question2 is None:
        return "To determine glaucoma screening indication, please provide the patient's age."

    age = patient_data.question2
    family_history_glaucoma = patient_data.question15 == "Item 2" if patient_data.question15 else None
    has_diabetes = patient_data.question5 == "Item 2" if patient_data.question5 else None # "Item 2" is "Yes" for diabetes question
    has_hypertension_medication = patient_data.question24 == "Item 1" if patient_data.question24 else None # "Item 1" is "Yes" for hypertension med

    if family_history_glaucoma is None:
        family_history_glaucoma = False # Assume no family history if not answered, for baseline recommendation.  Ideally, prompt for this.
        # return "To refine glaucoma screening recommendation, please provide information about family history of glaucoma."

    if has_diabetes is None:
        has_diabetes = False # Assume no diabetes if not answered, for baseline recommendation. Ideally, prompt for this.
        # return "To refine glaucoma screening recommendation, please provide information about diabetes status."

    if has_hypertension_medication is None:
        has_hypertension_medication = False # Assume no hypertension medication if not answered, for baseline recommendation. Ideally, prompt.
        # return "To refine glaucoma screening recommendation, please provide information about hypertension medication."


    risk_factors = []
    if family_history_glaucoma:
        risk_factors.append("family history of glaucoma")
    if has_diabetes:
        risk_factors.append("diabetes")
    if has_hypertension_medication:
        risk_factors.append("hypertension (medication use)")


    if risk_factors:
        risk_factor_message = "with risk factors: " + ", ".join(risk_factors)
    else:
        risk_factor_message = "without significant risk factors"

    frequency_message = ""
    base_frequency = "" # to store the base frequency for potential adjustments

    if age < 40:
        base_frequency = "every 5 to 10 years"
    elif 40 <= age <= 54:
        base_frequency = "every 2 to 4 years"
    elif 55 <= age <= 64:
        base_frequency = "every 1 to 3 years"
    elif age >= 65:
        base_frequency = "every 1 to 2 years"
    else:
        base_frequency = "based on ophthalmologist's discretion" # Should not reach here, but for safety.


    frequency = base_frequency # start with the base frequency

    # Adjust frequency if risk factors are present. Move to the next more frequent category if possible.
    if risk_factors:
        if base_frequency == "every 5 to 10 years":
            frequency = "every 2 to 4 years" # move to next level
        elif base_frequency == "every 2 to 4 years":
             frequency = "every 1 to 3 years" # move to next level
        elif base_frequency == "every 1 to 3 years":
            frequency = "every 1 to 2 years" # move to next level
        # "every 1 to 2 years" is already the most frequent in the average risk category, no further adjustment.


    modality = "Comprehensive eye examination by an ophthalmologist"

    return f"Patient is indicated for glaucoma screening {risk_factor_message}, modality: {modality}, frequency: {frequency}{frequency_message}."


def read_json_from_file(file_path: str) -> PatientData:
    """
    Reads patient data from a JSON file and converts it to PatientData dataclass.
    """
    with open(file_path, 'r', encoding='utf-8') as f:  # Specify encoding to handle potential character issues
        data = json.load(f)
    questionnaire_data = {}
    for page in data.get('pages', []):
        for element in page.get('elements', []):
            questionnaire_data[element['name']] = element.get('value')

    # map question names to PatientData fields, handling potential missing questions and type conversions.
    patient_data_dict = {}

    if 'question1' in questionnaire_data:
        # Handle cases where question1 is a single string or already a list
        q1_value = questionnaire_data['question1']
        if isinstance(q1_value, str):
            patient_data_dict['question1'] = [q1_value] # Convert to list if it's a single string
        elif isinstance(q1_value, list):
             patient_data_dict['question1'] = q1_value
        else:
            patient_data_dict['question1'] = None # or handle default as needed

    if 'question2' in questionnaire_data and questionnaire_data['question2'] is not None:
        try:
            patient_data_dict['question2'] = int(questionnaire_data['question2'])
        except ValueError:
            patient_data_dict['question2'] = None # Handle non-integer input if needed
    else:
        patient_data_dict['question2'] = None


    patient_data_dict['question15'] = questionnaire_data.get('question15') # handles missing keys automatically
    patient_data_dict['question5'] = questionnaire_data.get('question5')
    patient_data_dict['question24'] = questionnaire_data.get('question24')


    return PatientData(**patient_data_dict)


if __name__ == "__main__":
    file_path = 'questions.json'  # Assuming your json file is named questions.json in the same directory

    # Example usage with data loaded from JSON file
    #patient_data_from_json = read_json_from_file(file_path)
    #screening_recommendation = analyze_glaucoma_screening_indication(patient_data_from_json)
    #print(screening_recommendation)


    # Example usage with manual PatientData object (for testing different scenarios)
    # Example 1: 50 years old, family history of glaucoma, diabetes, hypertension
    patient_data_1 = PatientData(question1=["Item 12"], question2=50, question15="Item 2", question5="Item 2", question24="Item 1")
    recommendation_1 = analyze_glaucoma_screening_indication(patient_data_1)
    print(f"\nExample 1 Recommendation: {recommendation_1}")

    # Example 2: 68 years old, no family history, no diabetes, no hypertension
    patient_data_2 = PatientData(question1=["Item 12"], question2=68, question15="Item 1", question5="Item 1", question24="Item 2")
    recommendation_2 = analyze_glaucoma_screening_indication(patient_data_2)
    print(f"Example 2 Recommendation: {recommendation_2}")

    # Example 3: 35 years old, no family history, diabetes, hypertension
    patient_data_3 = PatientData(question1=["Item 12"], question2=35, question15="Item 1", question5="Item 2", question24="Item 1")
    recommendation_3 = analyze_glaucoma_screening_indication(patient_data_3)
    print(f"Example 3 Recommendation: {recommendation_3}")

    # Example 4: Missing Age
    patient_data_4 = PatientData(question1=["Item 12"], question2=None, question15="Item 1", question5="Item 2", question24="Item 1")
    recommendation_4 = analyze_glaucoma_screening_indication(patient_data_4)
    print(f"Example 4 Recommendation (Missing Age): {recommendation_4}")

    # Example 5: Not selected Glaucoma screening
    patient_data_5 = PatientData(question1=["Item 1"], question2=60, question15="Item 1", question5="Item 2", question24="Item 1") # Chose breast cancer but not glaucoma
    recommendation_5 = analyze_glaucoma_screening_indication(patient_data_5)
    print(f"Example 5 Recommendation (Not Selected Glaucoma): {recommendation_5}")

    # Example 6: Missing family history question
    patient_data_6 = PatientData(question1=["Item 12"], question2=60, question15=None, question5="Item 2", question24="Item 1")
    recommendation_6 = analyze_glaucoma_screening_indication(patient_data_6)
    print(f"Example 6 Recommendation (Missing Family History, but has other risks): {recommendation_6}")

    # Example 7: 45 years old, no family history, diabetes only
    patient_data_7 = PatientData(question1=["Item 12"], question2=45, question15="Item 1", question5="Item 2", question24="Item 2")
    recommendation_7 = analyze_glaucoma_screening_indication(patient_data_7)
    print(f"Example 7 Recommendation (Diabetes only): {recommendation_7}")

    # Example 8: 45 years old, no family history, hypertension only
    patient_data_8 = PatientData(question1=["Item 12"], question2=45, question15="Item 1", question5="Item 1", question24="Item 1")
    recommendation_8 = analyze_glaucoma_screening_indication(patient_data_8)
    print(f"Example 8 Recommendation (Hypertension only): {recommendation_8}")

    # Example 9: 45 years old, no family history, neither diabetes nor hypertension - base frequency
    patient_data_9 = PatientData(question1=["Item 12"], question2=45, question15="Item 1", question5="Item 1", question24="Item 2")
    recommendation_9 = analyze_glaucoma_screening_indication(patient_data_9)
    print(f"Example 9 Recommendation (Base Frequency): {recommendation_9}")


Example 1 Recommendation: Patient is indicated for glaucoma screening with risk factors: family history of glaucoma, diabetes, hypertension (medication use), modality: Comprehensive eye examination by an ophthalmologist, frequency: every 1 to 3 years.
Example 2 Recommendation: Patient is indicated for glaucoma screening without significant risk factors, modality: Comprehensive eye examination by an ophthalmologist, frequency: every 1 to 2 years.
Example 3 Recommendation: Patient is indicated for glaucoma screening with risk factors: diabetes, hypertension (medication use), modality: Comprehensive eye examination by an ophthalmologist, frequency: every 2 to 4 years.
Example 4 Recommendation (Missing Age): To determine glaucoma screening indication, please provide the patient's age.
Example 5 Recommendation (Not Selected Glaucoma): Patient is not indicated for glaucoma screening as it was not selected in requested screening.
Example 6 Recommendation (Missing Family History, but has othe